In [1]:
import argparse
import time
import shutil
from typing import List
import os
import queue

import numpy as np

import torch
from torch import Tensor
from torch.utils.tensorboard import SummaryWriter

from torchtext.data import Dataset

from joeynmt.batch import Batch
from joeynmt.helpers import log_data_info, load_config, log_cfg, \
    store_attention_plots, load_checkpoint, make_model_dir, \
    make_logger, set_seed, symlink_update, ConfigurationError
from joeynmt.model import Model
from joeynmt.prediction import validate_on_data
from joeynmt.loss import XentLoss
from joeynmt.data import load_data, make_data_iter
from joeynmt.builders import build_optimizer, build_scheduler, \
    build_gradient_clipper
from joeynmt.training import TrainManager
from joeynmt.prediction import test

In [2]:
import os
os.chdir('../')

In [3]:
from src.custom_model import build_model

In [4]:
from joeynmt.model import Model
from joeynmt.initialization import initialize_model
from joeynmt.embeddings import Embeddings
from joeynmt.encoders import Encoder, RecurrentEncoder, TransformerEncoder
from joeynmt.decoders import Decoder, RecurrentDecoder, TransformerDecoder
from joeynmt.constants import PAD_TOKEN, EOS_TOKEN, BOS_TOKEN
from joeynmt.search import beam_search, greedy
from joeynmt.vocabulary import Vocabulary
from joeynmt.batch import Batch
from joeynmt.helpers import ConfigurationError

In [5]:
def set_requires_grad(module, val):
    for p in module.parameters():
        p = p.detach()
        p.requires_grad = val

In [6]:

import argparse
import time
import shutil
from typing import List
import os
import queue

import math
import numpy as np
from logging import Logger

import torch
from torch import Tensor
from torch.utils.tensorboard import SummaryWriter

from torchtext.data import Dataset

from joeynmt.model import build_model
from joeynmt.batch import Batch
from joeynmt.helpers import log_data_info, load_config, log_cfg, \
    store_attention_plots, load_checkpoint, make_model_dir, \
    make_logger, set_seed, symlink_update, ConfigurationError
from joeynmt.model import Model
from joeynmt.prediction import validate_on_data
from joeynmt.loss import XentLoss
from joeynmt.data import load_data, make_data_iter
from joeynmt.builders import build_optimizer, build_scheduler, \
    build_gradient_clipper
from joeynmt.prediction import test

In [7]:
def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

def build_pretrained_model(cfg: dict = None,
                pretrained_model: Model = None,
                pretrained_src_vocab: Vocabulary = None,
                src_vocab: Vocabulary = None,
                trg_vocab: Vocabulary = None) -> Model:
    """
    Build and initialize the model according to the configuration.

    :param cfg: dictionary configuration containing model specifications
    :param src_vocab: source vocabulary
    :param trg_vocab: target vocabulary
    :return: built and initialized model
    """
    src_padding_idx = src_vocab.stoi[PAD_TOKEN]
    trg_padding_idx = trg_vocab.stoi[PAD_TOKEN]

    src_embed = Embeddings(
        **cfg["encoder"]["embeddings"], vocab_size=len(src_vocab),
        padding_idx=src_padding_idx)

    embedding_matrix = np.zeros((len(src_vocab), src_embed.embedding_dim))
    unknown_words = []
    for w in pretrained_src_vocab.itos:
        try:
            ix = pretrained_src_vocab.stoi[w]
            embedding_matrix[ix] = pretrained_model.src_embed.lut.weight[ix].cpu().detach().numpy()
        except KeyError:
            unknown_words.append(w)
    
    src_embed.lut.weight = torch.nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))

    trg_embed = Embeddings(
            **cfg["decoder"]["embeddings"], vocab_size=len(trg_vocab),
            padding_idx=trg_padding_idx)

    # build decoder
    dec_dropout = cfg["decoder"].get("dropout", 0.)
    dec_emb_dropout = cfg["decoder"]["embeddings"].get("dropout", dec_dropout)
    
    encoder = pretrained_model.encoder
    encoder.train()
    set_requires_grad(encoder, True)

    # build encoder
    #enc_dropout = cfg["encoder"].get("dropout", 0.)
    #enc_emb_dropout = cfg["encoder"]["embeddings"].get("dropout", enc_dropout)
    #if cfg["encoder"].get("type", "recurrent") == "transformer":
    #    assert cfg["encoder"]["embeddings"]["embedding_dim"] == \
    #           cfg["encoder"]["hidden_size"], \
    #           "for transformer, emb_size must be hidden_size"

    #    encoder = TransformerEncoder(**cfg["encoder"],
    #                                 emb_size=src_embed.embedding_dim,
    #                                 emb_dropout=enc_emb_dropout)
    #else:
    #    encoder = RecurrentEncoder(**cfg["encoder"],
    #                               emb_size=src_embed.embedding_dim,
    #                               emb_dropout=enc_emb_dropout)
    
    if cfg["decoder"].get("type", "recurrent") == "transformer":
        decoder = TransformerDecoder(
            **cfg["decoder"], encoder=encoder, vocab_size=len(trg_vocab),
            emb_size=trg_embed.embedding_dim, emb_dropout=dec_emb_dropout)
    else:
        decoder = RecurrentDecoder(
            **cfg["decoder"], encoder=encoder, vocab_size=len(trg_vocab),
            emb_size=trg_embed.embedding_dim, emb_dropout=dec_emb_dropout)

    model = Model(encoder=encoder, decoder=decoder,
                  src_embed=src_embed, trg_embed=trg_embed,
                  src_vocab=pretrained_model.src_vocab, trg_vocab=trg_vocab)

    # tie softmax layer with trg embeddings
    if cfg.get("tied_softmax", False):
        if trg_embed.lut.weight.shape == \
                model.decoder.output_layer.weight.shape:
            # (also) share trg embeddings and softmax layer:
            model.decoder.output_layer.weight = trg_embed.lut.weight
        else:
            raise ConfigurationError(
                "For tied_softmax, the decoder embedding_dim and decoder "
                "hidden_size must be the same."
                "The decoder must be a Transformer.")

    # custom initialization of model parameters
    initialize_model(model, cfg, src_padding_idx, trg_padding_idx)

    return model

In [8]:
cfg_file = "joeynmt/configs/sample_{name}.yaml".format(name="test")

In [9]:
def train_transfer(cfg_file: str) -> None:
    """
    Main training function. After training, also test on test data if given.

    :param cfg_file: path to configuration yaml file
    """
    cfg = load_config(cfg_file)

    # set the random seed
    set_seed(seed=cfg["pretraining"].get("random_seed", 42))

    # load the data
    pre_train_data, pre_dev_data, pre_test_data, pre_src_vocab, pre_trg_vocab = load_data(
        data_cfg=cfg["pretrained_data"])

    # build an encoder-decoder model
    pretrained_model = build_model(cfg["model"], src_vocab=pre_src_vocab, trg_vocab=pre_trg_vocab)

    # for training management, e.g. early stopping and model selection
    trainer = TrainManager(model=pretrained_model, config=cfg, training_key="pretraining")

    # store copy of original training config in model dir
    shutil.copy2(cfg_file, trainer.model_dir+"/config.yaml")

    # log all entries of config
    log_cfg(cfg, trainer.logger)

    log_data_info(train_data=pre_train_data, valid_data=pre_dev_data,
                  test_data=pre_test_data, src_vocab=pre_src_vocab, trg_vocab=pre_trg_vocab,
                  logging_function=trainer.logger.info)

    trainer.logger.info(str(pretrained_model))

    # store the vocabs
    src_vocab_file = "{}/src_vocab.txt".format(cfg["pretraining"]["model_dir"])
    pre_src_vocab.to_file(src_vocab_file)
    trg_vocab_file = "{}/trg_vocab.txt".format(cfg["pretraining"]["model_dir"])
    pre_trg_vocab.to_file(trg_vocab_file)

    # train the model
    trainer.train_and_validate(train_data=pre_train_data, valid_data=pre_dev_data)

    # predict with the best model on validation and test
    # (if test data is available)
    ckpt = "{}/{}.ckpt".format(trainer.model_dir, trainer.best_ckpt_iteration)
    output_name = "{:08d}.hyps".format(trainer.best_ckpt_iteration)
    output_path = os.path.join(trainer.model_dir, output_name)
    test(cfg_file, ckpt=ckpt, output_path=output_path, logger=trainer.logger,
        key_training="pretraining", key_data="pretrained_data")
    
    # set the random seed
    set_seed(seed=cfg["training"].get("random_seed", 42))

    # load the data
    train_data, dev_data, test_data, src_vocab, trg_vocab = load_data(
        data_cfg=cfg["data"])

    # build an encoder-decoder model
    model = build_pretrained_model(cfg["model"], 
                                   pretrained_model=pretrained_model, 
                                   pretrained_src_vocab=pre_src_vocab,
                                   src_vocab=src_vocab, 
                                   trg_vocab=trg_vocab)

    # for training management, e.g. early stopping and model selection
    trainer = TrainManager(model=model, config=cfg, training_key="training", logger=trainer.logger)

    # store copy of original training config in model dir
    shutil.copy2(cfg_file, trainer.model_dir+"/config.yaml")

    # log all entries of config
    log_cfg(cfg, trainer.logger)

    log_data_info(train_data=train_data, valid_data=dev_data,
                  test_data=test_data, src_vocab=src_vocab, trg_vocab=trg_vocab,
                  logging_function=trainer.logger.info)

    trainer.logger.info(str(model))

    # store the vocabs
    src_vocab_file = "{}/src_vocab.txt".format(cfg["training"]["model_dir"])
    src_vocab.to_file(src_vocab_file)
    trg_vocab_file = "{}/trg_vocab.txt".format(cfg["training"]["model_dir"])
    trg_vocab.to_file(trg_vocab_file)

    # train the model
    trainer.train_and_validate(train_data=train_data, valid_data=dev_data)

    # predict with the best model on validation and test
    # (if test data is available)
    ckpt = "{}/{}.ckpt".format(trainer.model_dir, trainer.best_ckpt_iteration)
    output_name = "{:08d}.hyps".format(trainer.best_ckpt_iteration)
    output_path = os.path.join(trainer.model_dir, output_name)
    test(cfg_file, ckpt=ckpt, output_path=output_path, logger=trainer.logger,
        key_training="training", key_data="data")

In [10]:
config = """
name: "my_experiment"

# This configuration serves the purpose of documenting and explaining the settings, *NOT* as an example for good hyperparamter settings.

data: # specify your data here
    src: {lang_src}                       # src language: expected suffix of train files, e.g. "train.de"
    trg: {lang_tgt}                       # trg language
    train: {train_path}     # training data
    dev: {dev_path}         # development data for validation
    test: {test_path}       # test data for testing final model; optional
    level: {level}                  # segmentation level: either "word", "bpe" or "char"
    lowercase: True                 # lowercase the data, also for validation
    max_sent_length: 150             # filter out longer sentences from training (src+trg)
    src_voc_min_freq: 1             # src minimum frequency for a token to become part of the vocabulary
    trg_voc_min_freq: 1             # trg minimum frequency for a token to become part of the vocabulary
    #src_vocab: "my_model/src_vocab.txt"  # if specified, load a vocabulary from this file
    #trg_vocab: "my_model/trg_vocab.txt"  # one token per line, line number is index

pretrained_data: # specify your data here
    src: {pretrained_lang_src}                       # src language: expected suffix of train files, e.g. "train.de"
    trg: {pretrained_lang_tgt}                       # trg language
    train: {pretrained_train_path}     # training data
    dev: {pretrained_dev_path}         # development data for validation
    test: {pretrained_test_path}       # test data for testing final model; optional
    level: {level}                  # segmentation level: either "word", "bpe" or "char"
    lowercase: True                 # lowercase the data, also for validation
    max_sent_length: 150             # filter out longer sentences from training (src+trg)
    src_voc_min_freq: 1             # src minimum frequency for a token to become part of the vocabulary
    trg_voc_min_freq: 1             # trg minimum frequency for a token to become part of the vocabulary
    #src_vocab: "my_model/src_vocab.txt"  # if specified, load a vocabulary from this file
    #trg_vocab: "my_model/trg_vocab.txt"  # one token per line, line number is index

testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 5                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search

training:                           # specify training details here
    reset_best_ckpt: False          # if True, reset the tracking of the best checkpoint and scores. Use for domain adaptation or fine-tuning with new metrics or dev data.
    reset_scheduler: False          # if True, overwrite scheduler in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    reset_optimizer: False          # if True, overwrite optimizer in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    learning_rate: 0.0005           # initial learning rate, default: 3.0e-4 / 0.005
    learning_rate_min: 0.0001       # stop learning when learning rate is reduced below this threshold, default: 1.0e-8
    #learning_rate_factor: 1        # factor for Noam scheduler (used with Transformer)
    #learning_rate_warmup: 4000     # warmup steps for Noam scheduler (used with Transformer)
    clip_grad_val: 1.0              # clip the gradients to this value when they exceed it, optional
    #clip_grad_norm: 1.0            # norm clipping instead of value clipping
    weight_decay: 0.                # l2 regularization, default: 0
    batch_size: 48                  # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token")
    batch_type: "sentence"          # create batches with sentences ("sentence", default) or tokens ("token")
    eval_batch_size: 10            # mini-batch size for evaluation (see batch_size above)
    eval_batch_type: "sentence"     # evaluation batch type ("sentence", default) or tokens ("token")
    batch_multiplier: 1             # increase the effective batch size with values >1 to batch_multiplier*batch_size without increasing memory consumption by making updates only every batch_multiplier batches
    scheduling: "plateau"           # learning rate scheduling, optional, if not specified stays constant, options: "plateau", "exponential", "decaying", "noam" (for Transformer), "warmupexponentialdecay"
    patience: 600                     # specific to plateau scheduler: wait for this many validations without improvement before decreasing the learning rate
    decrease_factor: 0.5            # specific to plateau & exponential scheduler: decrease the learning rate by this factor
    epochs: 1                      # train for this many epochs
    validation_freq: {val_freq}            # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 1000               # log the training progress after this many updates, default: 100
    eval_metric: "bleu"             # validation metric, default: "bleu", other options: "chrf", "token_accuracy", "sequence_accuracy"
    early_stopping_metric: "eval_metric"   # when a new high score on this metric is achieved, a checkpoint is written, when "eval_metric" (default) is maximized, when "loss" or "ppl" is minimized
    model_dir: {model_dir} # directory where models and validation results are stored, required
    overwrite: True                 # overwrite existing model directory, default: False. Do not set to True unless for debugging!
    shuffle: True                   # shuffle the training data, default: True
    use_cuda: False                  # use CUDA for acceleration on GPU, required. Set to False when working on CPU.
    max_output_length: 60           # maximum output length for decoding, default: None. If set to None, allow sentences of max 1.5*src length
    print_valid_sents: []    # print this many validation sentences during each validation run, default: [0, 1, 2]
    keep_last_ckpts: 3              # keep this many of the latest checkpoints, if -1: all of them, default: 5
    label_smoothing: 0.0            # label smoothing: reference tokens will have 1-label_smoothing probability instead of 1, rest of probability mass is uniformly distributed over the rest of the vocabulary, default: 0.0 (off)

pretraining:                           # specify training details here
    reset_best_ckpt: False          # if True, reset the tracking of the best checkpoint and scores. Use for domain adaptation or fine-tuning with new metrics or dev data.
    reset_scheduler: False          # if True, overwrite scheduler in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    reset_optimizer: False          # if True, overwrite optimizer in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    learning_rate: 0.0002           # initial learning rate, default: 3.0e-4 / 0.005
    learning_rate_min: 0.00001       # stop learning when learning rate is reduced below this threshold, default: 1.0e-8
    #learning_rate_factor: 1        # factor for Noam scheduler (used with Transformer)
    #learning_rate_warmup: 4000     # warmup steps for Noam scheduler (used with Transformer)
    clip_grad_val: 1.0              # clip the gradients to this value when they exceed it, optional
    #clip_grad_norm: 1.0            # norm clipping instead of value clipping
    weight_decay: 0.                # l2 regularization, default: 0
    batch_size: 48                  # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token")
    batch_type: "sentence"          # create batches with sentences ("sentence", default) or tokens ("token")
    eval_batch_size: 10            # mini-batch size for evaluation (see batch_size above)
    eval_batch_type: "sentence"     # evaluation batch type ("sentence", default) or tokens ("token")
    batch_multiplier: 1             # increase the effective batch size with values >1 to batch_multiplier*batch_size without increasing memory consumption by making updates only every batch_multiplier batches
    scheduling: "plateau"           # learning rate scheduling, optional, if not specified stays constant, options: "plateau", "exponential", "decaying", "noam" (for Transformer), "warmupexponentialdecay"
    patience: 600                     # specific to plateau scheduler: wait for this many validations without improvement before decreasing the learning rate
    decrease_factor: 0.5            # specific to plateau & exponential scheduler: decrease the learning rate by this factor
    epochs: 1                      # train for this many epochs
    validation_freq: {val_freq}            # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 1000               # log the training progress after this many updates, default: 100
    eval_metric: "bleu"             # validation metric, default: "bleu", other options: "chrf", "token_accuracy", "sequence_accuracy"
    early_stopping_metric: "eval_metric"   # when a new high score on this metric is achieved, a checkpoint is written, when "eval_metric" (default) is maximized, when "loss" or "ppl" is minimized
    model_dir: {model_dir} # directory where models and validation results are stored, required
    overwrite: True                 # overwrite existing model directory, default: False. Do not set to True unless for debugging!
    shuffle: True                   # shuffle the training data, default: True
    use_cuda: False                  # use CUDA for acceleration on GPU, required. Set to False when working on CPU.
    max_output_length: 60           # maximum output length for decoding, default: None. If set to None, allow sentences of max 1.5*src length
    print_valid_sents: []    # print this many validation sentences during each validation run, default: [0, 1, 2]
    keep_last_ckpts: 3              # keep this many of the latest checkpoints, if -1: all of them, default: 5
    label_smoothing: 0.0            # label smoothing: reference tokens will have 1-label_smoothing probability instead of 1, rest of probability mass is uniformly distributed over the rest of the vocabulary, default: 0.0 (off)

model:                              # specify your model architecture here
    initializer: "xavier"           # initializer for all trainable weights (xavier, zeros, normal, uniform)
    init_weight: 0.01               # weight to initialize; for uniform, will use [-weight, weight]
    init_gain: 1.0                  # gain for Xavier initializer (default: 1.0)
    bias_initializer: "zeros"       # initializer for bias terms (xavier, zeros, normal, uniform)
    embed_initializer: "normal"     # initializer for embeddings (xavier, zeros, normal, uniform)
    embed_init_weight: 0.1          # weight to initialize; for uniform, will use [-weight, weight]
    embed_init_gain: 1.0            # gain for Xavier initializer for embeddings (default: 1.0)
    init_rnn_orthogonal: False      # use orthogonal initialization for recurrent weights (default: False)
    lstm_forget_gate: 1.            # initialize LSTM forget gate with this value (default: 1.)
    tied_embeddings: False           # tie src and trg embeddings, only applicable if vocabularies are the same, default: False
    tied_softmax: False             # tie trg embeddings and softmax (for Transformer; can be used together with tied_embeddings), default: False
    encoder:
        type: "recurrent"           # encoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        rnn_type: "gru"             # type of recurrent unit to use, either "gru" or "lstm", default: "lstm"
        embeddings:
            embedding_dim: {emb_size}      # size of embeddings
            scale: False            # scale the embeddings by sqrt of their size, default: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: {hidden_size}            # size of RNN
        bidirectional: True         # use a bi-directional encoder, default: True
        dropout: 0.3                # apply dropout to the inputs to the RNN, default: 0.0
        num_layers: 2               # stack this many layers of equal size, default: 1
        freeze: False               # if True, encoder parameters are not updated during training (does not include embedding parameters)
    decoder:
        type: "recurrent"           # decoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        rnn_type: "gru"
        embeddings:
            embedding_dim: {emb_size}
            scale: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: {hidden_size}
        dropout: 0.3
        hidden_dropout: 0.2         # apply dropout to the attention vector, default: 0.0
        num_layers: 2
        input_feeding: True         # combine hidden state and attention vector before feeding to rnn, default: True
        init_hidden: "last"         # initialized the decoder hidden state: use linear projection of last encoder state ("bridge") or simply the last state ("last") or zeros ("zero"), default: "bridge"
        attention: "bahdanau"       # attention mechanism, choices: "bahdanau" (MLP attention), "luong" (bilinear attention), default: "bahdanau"
        freeze: False               # if True, decoder parameters are not updated during training (does not include embedding parameters, but attention)
"""

In [11]:
f_config = config.format(lang_src='es', lang_tgt='shp', 
                    train_path=os.path.join('data/translate/preprocessed/Religioso/word', 'train'),
                    test_path=os.path.join('data/translate/preprocessed/Religioso/word', 'test'),
                    dev_path=os.path.join('data/translate/preprocessed/Religioso/word', 'valid'),
                    pretrained_lang_src='es', pretrained_lang_tgt='shp', 
                    pretrained_train_path=os.path.join('data/translate/preprocessed/Educativo/word', 'train'),
                    pretrained_test_path=os.path.join('data/translate/preprocessed/Educativo/word', 'test'),
                    pretrained_dev_path=os.path.join('data/translate/preprocessed/Educativo/word', 'valid'),
                    level='word',
                    emb_size=2,
                    hidden_size=5,
                    val_freq=2,
                    model_dir=os.path.join('results/temp'))


In [12]:
with open("joeynmt/configs/sample_{name}.yaml".format(name="test"),'w') as f:
    f.write(f_config)

In [17]:
train("joeynmt/configs/sample_{name}.yaml".format(name="test"))

2020-02-26 17:44:34,454 Hello! This is Joey-NMT.
2020-02-26 17:44:35,590 Total params: 31891
2020-02-26 17:44:35,591 Trainable parameters: ['decoder.att_vector_layer.bias', 'decoder.att_vector_layer.weight', 'decoder.attention.energy_layer.weight', 'decoder.attention.key_layer.weight', 'decoder.attention.query_layer.weight', 'decoder.output_layer.weight', 'decoder.rnn.bias_hh_l0', 'decoder.rnn.bias_hh_l1', 'decoder.rnn.bias_ih_l0', 'decoder.rnn.bias_ih_l1', 'decoder.rnn.weight_hh_l0', 'decoder.rnn.weight_hh_l1', 'decoder.rnn.weight_ih_l0', 'decoder.rnn.weight_ih_l1', 'encoder.rnn.bias_hh_l0', 'encoder.rnn.bias_hh_l0_reverse', 'encoder.rnn.bias_hh_l1', 'encoder.rnn.bias_hh_l1_reverse', 'encoder.rnn.bias_ih_l0', 'encoder.rnn.bias_ih_l0_reverse', 'encoder.rnn.bias_ih_l1', 'encoder.rnn.bias_ih_l1_reverse', 'encoder.rnn.weight_hh_l0', 'encoder.rnn.weight_hh_l0_reverse', 'encoder.rnn.weight_hh_l1', 'encoder.rnn.weight_hh_l1_reverse', 'encoder.rnn.weight_ih_l0', 'encoder.rnn.weight_ih_l0_reve